In [ ]:
# ===============================
# Kidney Classification - Colab Training using Folder Names
# ===============================

import os
import numpy as np
from PIL import Image
from sklearn.model_selection import train_test_split
from tensorflow.keras import layers, models, optimizers
from tensorflow.keras.utils import to_categorical
from sklearn.metrics import classification_report
import matplotlib.pyplot as plt
from google.colab import drive

# -------------------------
# 1. Mount Google Drive
# -------------------------
drive.mount('/content/drive')

# -------------------------
# 2. Paths and Config
# -------------------------
DATASET_ROOT = "/content/drive/MyDrive/Data/CT-KIDNEY-DATASET-Normal-Cyst-Tumor-Stone"
IMAGE_HEIGHT, IMAGE_WIDTH = 128, 128
EPOCHS = 10
BATCH_SIZE = 32
LEARNING_RATE = 1e-3
TEST_SIZE = 0.2

# -------------------------
# 3. Load Images
# -------------------------
print("Loading dataset...")
X, y, class_names = [], [], []

for idx, class_name in enumerate(sorted(os.listdir(DATASET_ROOT))):
    class_path = os.path.join(DATASET_ROOT, class_name)
    if not os.path.isdir(class_path):
        continue
    class_names.append(class_name)
    for fname in os.listdir(class_path):
        fpath = os.path.join(class_path, fname)
        if not os.path.isfile(fpath):
            continue
        if os.path.splitext(fpath)[1].lower() not in [".jpg", ".jpeg", ".png", ".bmp", ".webp"]:
            continue
        try:
            img = Image.open(fpath).convert("RGB")
            img = img.resize((IMAGE_WIDTH, IMAGE_HEIGHT))
            X.append(np.array(img, dtype=np.float32) / 255.0)
            y.append(idx)
        except:
            continue

X = np.array(X, dtype=np.float32)
y = np.array(y, dtype=np.int64)
print(f"Dataset loaded: {X.shape[0]} images, Classes: {class_names}")

# -------------------------
# 4. Split Data
# -------------------------
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=TEST_SIZE, stratify=y, random_state=42
)
print(f"Train: {X_train.shape}, Test: {X_test.shape}")

# -------------------------
# 5. Model
# -------------------------
model = models.Sequential([
    layers.Input(shape=(IMAGE_HEIGHT, IMAGE_WIDTH, 3)),
    layers.Conv2D(32, (3,3), activation="relu", padding="same"),
    layers.MaxPooling2D((2,2)),
    layers.Conv2D(64, (3,3), activation="relu", padding="same"),
    layers.MaxPooling2D((2,2)),
    layers.Conv2D(128, (3,3), activation="relu", padding="same"),
    layers.MaxPooling2D((2,2)),
    layers.Dropout(0.3),
    layers.Flatten(),
    layers.Dense(256, activation="relu"),
    layers.Dropout(0.4),
    layers.Dense(len(class_names), activation="softmax")
])

model.compile(
    optimizer=optimizers.Adam(learning_rate=LEARNING_RATE),
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"]
)

model.summary()

# -------------------------
# 6. Train
# -------------------------
history = model.fit(
    X_train, y_train,
    validation_split=0.1,
    epochs=EPOCHS,
    batch_size=BATCH_SIZE,
    shuffle=True,
    verbose=1
)

# -------------------------
# 7. Evaluate
# -------------------------
test_loss, test_acc = model.evaluate(X_test, y_test, verbose=0)
print(f"\nTest Accuracy: {test_acc:.4f}")

y_pred = np.argmax(model.predict(X_test), axis=1)
print("\nClassification Report:\n")
print(classification_report(y_test, y_pred, target_names=class_names))

# -------------------------
# 8. Save Model
# -------------------------
model_save_path = "/content/drive/MyDrive/Data/kidney_cnn.h5"
model.save(model_save_path)
print(f"Model saved to {model_save_path}")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Loading dataset...


In [8]:
import os

DATASET_ROOT = "/content/drive/MyDrive/Data/CT-KIDNEY-DATASET-Normal-Cyst-Tumor-Stone"

for folder in sorted(os.listdir(DATASET_ROOT)):
    path = os.path.join(DATASET_ROOT, folder)
    if os.path.isdir(path):
        num_files = len([f for f in os.listdir(path) if f.lower().endswith((".jpg",".jpeg",".png"))])
        print(f"{folder}: {num_files} images")


Cyst: 0 images
Normal: 0 images
Stone: 0 images
Tumor: 924 images
